In [9]:
import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.operations import SymmOp
from jupyter_jsmol.pymatgen import quick_view

In [10]:
# Clé API et ID Matériau
mp_key = "71t7m37H7F5Jw2PfGOrlKxgH4ZbRQh3W"
mp_id = "mp-11739"

In [26]:
# Chargement de la structure depuis Materials Project
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
# Conversion en structure conventionnelle
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()

# Analyseur de groupe d'espace
sga = SpacegroupAnalyzer(conv_struc)
sym_ops = sga.get_symmetry_operations()

In [28]:
# Fonction pour afficher la structure
def show_structure(structure):
    view = quick_view(structure, "packed", conventional=True)
    display(view)
    return view

In [29]:
# Affichage de la structure de base
view=show_structure(conv_struc)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [30]:
# Sélection des opérations de symétrie spécifiques
inversion_op = sym_ops[2]  # Inversion
rotation_op = sym_ops[5]  # Rotation d'ordre 3
rotoinversion_op = sym_ops[6]  # Roto-inversion

In [31]:
# Atomes choisis pour l'application des symétries
atom_indices = {"K": 0, "La": 1, "Te1": 2, "Te2": 3}

In [33]:
# Application et affichage des résultats
for label, (op, atom_index) in zip([
    "Inversion", "Rotation d'ordre 3", "Roto-inversion"
], [(inversion_op, atom_indices["Te1"]), (rotation_op, atom_indices["Te2"]), (rotoinversion_op, atom_indices["K"])]):
    
    pos_init = prim_struc.sites[atom_index].frac_coords
    pos_final = op.operate(pos_init)
    
    # Trouver l'atome correspondant après transformation
    j_atom = None
    for k_atom, site in enumerate(prim_struc.sites):
        if np.linalg.norm((site.frac_coords - pos_final) % 1) < 1e-6:
            j_atom = k_atom
            break
    
    print(f"{label} sur {prim_struc.sites[atom_index].species_string} (Atome {atom_index+1}):")
    print("Rot:")
    print(op.rotation_matrix)
    print("tau")
    print(op.translation_vector)
    print(f"  Position initiale: {pos_init}")
    print(f"  Position finale: {pos_final}")
    if j_atom is not None:
        print(f"  Correspondance trouvée avec l'atome {j_atom+1} ({prim_struc.sites[j_atom].species_string}) à {prim_struc.sites[j_atom].frac_coords}")
    print()
    
    # Affichage dans JSmol
    view.script(f'draw SYMOP {sym_ops.index(op)+1} {{atomno = {atom_index+1}}}')

Inversion sur Te (Atome 3):
Rot:
[[ 0. -1.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
  Position initiale: [0.26073622 0.26073622 0.26073622]
  Position finale: [-0.26073622  0.          0.26073622]

Rotation d'ordre 3 sur Te (Atome 4):
Rot:
[[ 1. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
  Position initiale: [0.73926378 0.73926378 0.73926378]
  Position finale: [ 0.          0.73926378 -0.73926378]

Roto-inversion sur K (Atome 1):
Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
  Position initiale: [0.5 0.5 0.5]
  Position finale: [ 0.5  0.5 -0.5]
  Correspondance trouvée avec l'atome 1 (K) à [0.5 0.5 0.5]



In [34]:
# Affichage de la structure avec JSmol
def show_symmetry_op(op, atom_index):
    view = show_structure(prim_struc)
    view.script(f'draw SYMOP {sym_ops.index(op)+1} {{atomno = {atom_index+1}}}')
    return view

# Création de cellules de visualisation pour chaque symétrie
for op, atom_index in [(inversion_op, atom_indices["Te1"]), (rotation_op, atom_indices["Te2"]), (rotoinversion_op, atom_indices["K"])]:
    show_symmetry_op(op, atom_index)


JsmolView(layout=Layout(align_self='stretch', height='400px'))

JsmolView(layout=Layout(align_self='stretch', height='400px'))

JsmolView(layout=Layout(align_self='stretch', height='400px'))